In [1]:
import tensorflow as tf

2023-04-23 22:36:51.500989: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-23 22:36:51.528437: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-23 22:36:51.528691: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-23 22:36:52.089740: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler, LabelEncoder
from joblib import dump, load

In [3]:
NB_CLIENTS = 9
TEST_CLIENT = None

DATA_PATH_PATTERN = "../data/Device #%d.csv"
PRETRAINED_MODEL = "../models/dl.h5"

WHOLE_CLASSES = ['benign', 'm_scan', 'm_ack', 'm_syn', 'm_udp', 'm_udpplain', 'g_combo', 'g_junk', 'g_scan', 'g_tcp', 'g_udp']
BINARY_MODE = False
BATCH_SIZE = 64
NB_CLASSES = len(WHOLE_CLASSES)
NB_EPOCHES = 1
NB_ROUNDS = 100
MX_RECORDS = 100000

In [4]:
global le, sc

sc = load('../models/std_scaler.bin')
le = load('../models/lab_encoder.bin')


def preprocessor(_client_data: pd.DataFrame):
    global sc, le
    _client_data = shuffle(_client_data)
    _x = _client_data.drop(["type"], axis=1)
    _y = _client_data["type"]
    _x = sc.transform(_x)
    
    if BINARY_MODE:
        _y = (_y == "benign").astype(int)
    else:
        _y = le.transform(_y)
        _y = tf.keras.utils.to_categorical(_y, num_classes=NB_CLASSES)
        
    return _x, _y


def make_dataset(_x, _y):
    return tf.data.Dataset.from_tensor_slices(
        collections.OrderedDict(
            x=tf.constant(_x),
            y=tf.constant(_y)
        )
    )


def client_data(_x, _y):
    _client_data = make_dataset(_x, _y)
    return _client_data.shuffle(MX_RECORDS).batch(BATCH_SIZE).repeat(NB_EPOCHES)

In [5]:
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, input_dim = 116),
        tf.keras.layers.GroupNormalization(),
#         tf.keras.layers.BatchNormalization(synchronized=True, input_dim=116),
#         tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(64),
        tf.keras.layers.LeakyReLU(alpha=0.05),
        tf.keras.layers.Dense(32),
        tf.keras.layers.LeakyReLU(alpha=0.05),
        tf.keras.layers.Dense(32),
        tf.keras.layers.LeakyReLU(alpha=0.05),
        tf.keras.layers.Dense(NB_CLASSES, activation = "softmax")
    ])
    model.compile('rmsprop', 'categorical_crossentropy', metrics=['acc'])
    model.load_weights(PRETRAINED_MODEL)
    return model

In [6]:
model = create_model()

In [7]:
raw_clients_data = []
test_raw_data = None

for idx in range(NB_CLIENTS):
    raw_content = shuffle(pd.read_csv(DATA_PATH_PATTERN % (idx + 1)))
    x_test, y_test = preprocessor(raw_content)
    print(model.evaluate(x_test, y_test))

/opt/Anaconda310/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


31822/31822 [==============================] - 17s 529us/step - loss: 0.1388 - acc: 0.9086
[0.1387580931186676, 0.9085562229156494]


/opt/Anaconda310/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


26122/26122 [==============================] - 14s 535us/step - loss: 0.0560 - acc: 0.9938
[0.055956874042749405, 0.9937741756439209]


/opt/Anaconda310/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


...

11726/11726 [==============================] - 6s 530us/step - loss: 0.1555 - acc: 0.9880
[0.15552304685115814, 0.9879751205444336]


/opt/Anaconda310/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


26971/26971 [==============================] - 15s 545us/step - loss: 0.1143 - acc: 0.9611
[0.11430574953556061, 0.9611045122146606]


/opt/Anaconda310/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


12957/12957 [==============================] - 7s 538us/step - loss: 0.0112 - acc: 0.9987
[0.01115789357572794, 0.9986566305160522]
